## Imports

In [2]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import pprint
import json
import re
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

/tmp/ipykernel_173924/3699664375.py:9: DeprecationWarning: Please import `gaussian_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter


## Make New Folders

In [ ]:
File_NCs = ['NE_002', 'NE_003', 'NE_004', 'NE_005', 'E_071', 'E_068', 'NE_006', 'NE_007', 'NE_008', 'NE_009', 'NE_010', 'NE_011', 'NE_012', 'NE_013',
            'NE_014', 'NE_015', 'NE_016', 'NE_017', 'NE_018', 'NE_019', 'NE_020', 'NE_022', 'NE_024', 'NE_025', 'E_038', 'NE_027', 'NE_029', 'NE_030', 
            'NE_031', 'NE_032', 'NE_033', 'NE_034', 'NE_036', 'NE_037', 'NE_039', 'NE_040', 'NE_041', 'NE_042', 'NE_043', 'NE_044', 'NE_045', 'NE_046', 
            'NE_047', 'NE_048', 'NE_051', 'NE_052', 'NE_053', 'NE_054', 'NE_056', 'NE_057', 'NE_058', 'NE_059', 'NE_060', 'NE_061', 'NE_062', 'NE_063', 
            'NE_064', 'NE_065', 'E_067', 'E_069']


participant_path = '*** REPLACE WITH YOUR PATH! ***'

for index, File_NC in enumerate(File_NCs):
    new_dir = f'{participant_path}/{File_NC}/Fixations/Fixation_Sequences'
    os.makedirs(new_dir, exist_ok=True)


## Helper Functions

In [3]:
def contains_fragment(text, fragment):
  match = re.search(fragment, text)
  if match:
    return True
  else:
    return False

## Add Unix Timestamps to Fixation File

In [5]:
## Need to update these for each new participant
for participant_NC in os.listdir(participant_path):
  try:
    ## Initial Phase
    fixation_file = f"{participant_path}/{participant_NC}/80_MS/fixations.csv"
    info_player_file = f'{participant_path}/{participant_NC}/80_MS/info.player.json'

    Fin = open(info_player_file)
    jsondata = json.load(Fin)
    Fin.close()

    start_time_system = jsondata["start_time_system_s"]
    start_time_synced = jsondata["start_time_synced_s"]
    start_timestamp_diff = start_time_system - start_time_synced

    fixations_df = pd.read_csv(fixation_file)
    fixations_df["start_timestamp_unix"] = fixations_df["start_timestamp"] + start_timestamp_diff
    fixations_df["start_timestamp_datetime"] = pd.to_datetime(fixations_df["start_timestamp_unix"], unit="s")
    fixations_df.to_csv(f'{participant_path}/{participant_NC}/80_MS/fixations_unix_datetime.csv')

  except:
    print(f"Trouble Processing {participant_NC}")
  print(f"\nProcessing {participant_NC}: DONE")


Processing E_038: DONE
Trouble Processing Pol_E01

Processing Pol_E01: DONE

Processing NE_017: DONE

Processing NE_034: DONE

Processing NE_012: DONE

Processing NE_061: DONE

Processing NE_006: DONE

Processing NE_046: DONE

Processing NE_052: DONE
Trouble Processing E_071

Processing E_071: DONE

Processing NE_019: DONE

Processing NE_062: DONE

Processing NE_027: DONE

Processing E_072: DONE

Processing NE_053: DONE

Processing E_067: DONE

Processing NE_031: DONE

Processing NE_054: DONE

Processing NE_025: DONE

Processing NE_013: DONE

Processing NE_007: DONE

Processing NE_058: DONE

Processing NE_039: DONE

Processing NE_065: DONE

Processing E_026: DONE

Processing NE_036: DONE

Processing NE_051: DONE

Processing NE_002: DONE

Processing NE_008: DONE

Processing NE_057: DONE

Processing NE_022: DONE

Processing NE_043: DONE

Processing NE_024: DONE

Processing NE_041: DONE

Processing NE_033: DONE

Processing NE_010: DONE
Trouble Processing MSU_E01

Processing MSU_E01: DONE

## Create Individual Fixation Files

In [ ]:
## Need to update these for each new participant
for participant_NC in os.listdir(participant_path):
  try:
    print(f'Processing {participant_NC} ...')
    jsonFile = f'PATH/{participant_NC}_comprehensive_2.json'
    Fin = open(jsonFile)
    jsondata = json.load(Fin)
    Fin.close()

    ## Grab the Batch Number so the image list can be extracted
    batch_num = f'batch{jsondata[0]["batch"]}'
    imageListPath = f'PATH/{batch_num}_images.csv'
    imageURLS = []

    with open(imageListPath, 'r') as f:
      for line in f:
          imageURLS.append(line.strip())
      f.close()

    ## Initial Phase
    fixation_file = f"{participant_path}/{participant_NC}/80_MS/fixations_unix_datetime.csv"
    output_dir = f"{participant_path}/{participant_NC}/Fixations/Initial_Phase/"

    # Load the master Gaze file
    master_df = pd.read_csv(fixation_file)

    for index, jsonObject in enumerate(jsondata):

      if index % 9 == 0:
        print("Processing ...")

      start_index = int(jsonObject["imageStartIndex"])
      end_index = int(jsonObject["initialDecisionIndex"])

      url = imageURLS[index]

      image_ID = os.path.splitext(url)[0]

      # Filter rows where world_index is within the specified range
      filtered_df = master_df[
          (master_df["end_frame_index"] >= start_index) & (master_df["start_frame_index"] <= end_index)
      ]

      # Save the filtered data to a new CSV file
      output_filename = f"{output_dir}/{participant_NC}_{image_ID}_Fixations.csv"
      filtered_df.to_csv(output_filename, index=False)

  except:
    print(f"Trouble Processing {participant_NC}")
  print(f"\nProcessing {participant_NC}: DONE")

Processing E_038 ...
Trouble Processing E_038

Processing E_038: DONE
Processing Pol_E01 ...
Trouble Processing Pol_E01

Processing Pol_E01: DONE
Processing NE_017 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

Processing NE_017: DONE
Processing NE_034 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

Processing NE_034: DONE
Processing NE_012 ...
Trouble Processing NE_012

Processing NE_012: DONE
Processing NE_061 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

Processing NE_061: DONE
Processing NE_006 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

Processing NE_006: DONE
Processing NE_046 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

Processing NE_046: DONE
Processing NE_052 ...
Processing ...
Processing ...
Processing ...
Processing ...
Processing ...

## Concatenate Files

In [ ]:
attackTypes = ["StyleGAN2", "StyleGAN3", "Diseased", "Textured Contact", "Contacts + Print", "Real Iris","Printout", "Artificial", "Post-Mortem", 
                "Synthetic", "Glass Prosthesis / Artificial"]

fragment = "/fbi_images_unblurred/"

for attackType in attackTypes:
    for participant_NC in os.listdir(participant_path):
        try:
            master_df = pd.read_csv(f"{participant_path}/{participant_NC}/80_MS/fixations_unix_datetime.csv")
            print(f'Processing {participant_NC} ...')
            jsonFile = f'PATH/{participant_NC}_comprehensive_2.json'
            Fin = open(jsonFile)
            jsondata = json.load(Fin)
            Fin.close()

            fixationFiles = []

            for index, jsonObject in enumerate(jsondata):
                if jsonObject["attackType"] == attackType:
                    imageName = jsonObject["irisImageLink"]

                    if contains_fragment(jsonObject["irisImageLink"], fragment):
                        imageName = re.sub(fragment,"",imageName)
                        imageName = os.path.splitext(imageName)[0]

                    if attackType == "StyleGAN2":
                        imageName = os.path.splitext(imageName)[0:1]
                        fileLink = f'{participant_path}/{participant_NC}/Fixations/Initial_Phase/{participant_NC}_{imageName[0]}_Fixations.csv'

                    elif attackType == "StyleGAN3":
                        imageName = os.path.splitext(imageName)[0:1]
                        fileLink = f'{participant_path}/{participant_NC}/Fixations/Initial_Phase/{participant_NC}_{imageName[0]}_Fixations.csv'
                    else:
                        fileLink = f'{participant_path}/{participant_NC}/Fixations/Initial_Phase/{participant_NC}_{imageName}_Fixations.csv'

                    fixationFiles.append(fileLink)

            df_list = [pd.read_csv(fixationFile) for fixationFile in fixationFiles]
            merged_df = pd.concat(df_list, ignore_index=True)
            merged_df['start_timestamp_unix'] = pd.to_numeric(merged_df['start_timestamp_unix'], errors='coerce')  # Convert to numeric if not already
            merged_df = merged_df.sort_values(by='start_timestamp_unix')

            if attackType != "Glass Prosthesis / Artificial":
                merged_df.to_csv(f'{participant_path}/{participant_NC}/Fixations/{attackType}_merged_output.csv', index=False)
            else:
                merged_df.to_csv(f'{participant_path}/{participant_NC}/Fixations/Glass_Prosthesis_merged_output.csv', index=False)
        except:
            print(f'Encountered Problem Processing Participant: {participant_NC}')


Processing E_038 ...
Encountered Problem Processing Participant: Pol_E01
Processing NE_017 ...
Processing NE_034 ...
Processing NE_012 ...
Processing NE_061 ...
Processing NE_006 ...
Processing NE_046 ...
Processing NE_052 ...
Encountered Problem Processing Participant: E_071
Processing NE_019 ...
Processing NE_062 ...
Processing NE_027 ...
Processing E_072 ...
Encountered Problem Processing Participant: E_072
Processing NE_053 ...
Processing E_067 ...
Processing NE_031 ...
Processing NE_054 ...
Processing NE_025 ...
Processing NE_013 ...
Processing NE_007 ...
Processing NE_058 ...
Processing NE_039 ...
Processing NE_065 ...
Processing E_026 ...
Processing NE_036 ...
Processing NE_051 ...
Processing NE_002 ...
Processing NE_008 ...
Processing NE_057 ...
Processing NE_022 ...
Processing NE_043 ...
Processing NE_024 ...
Processing NE_041 ...
Encountered Problem Processing Participant: NE_041
Processing NE_033 ...
Processing NE_010 ...
Encountered Problem Processing Participant: MSU_E01
P